In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [ ]:
df_bilt = pd.read_csv('Data/de_bilt_weather.csv')
df_bilt['days'] = df_bilt['days'] - 1
df_bilt2000 = df_bilt.loc[df_bilt['year'] == 2000].copy()
df_bilt2001 = df_bilt.loc[df_bilt['year'] == 2001].copy()

variables = ['cloud_cover', 'wind_speed', 'wind_gust', 'humidity',          # george
            'pressure', 'global_radiation', 'precipitation', 'sunshine',    # skipper
            'temp_mean', 'temp_min', 'temp_max']                            # thijs

n_var = len(variables)

temp_data = np.array(df_bilt2000['temp_mean'][:30])
temp_data

In [ ]:
k_array = np.arange(10)
t_array = np.arange(1, 10)

np.outer(k_array, t_array)

In [ ]:
def DFT(data):
    N = len(data)
    
    k_array = np.arange(N)
    t_array = np.arange(N)

    exponentials =  np.exp(-2j * np.pi * np.outer(k_array, t_array) / N)
    coefs = exponentials @ data
    
    return k_array, coefs

def predict(t_array, k_array, coefs):
    N = len(coefs)
    prediction = np.zeros(len(t_array))

    for k in k_array:
        A = np.abs(coefs[k]) / N
        phase = np.angle(coefs[k])
        prediction += A * np.cos(phase + 2 * np.pi * k * t_array / N)

    return prediction

k, coefs = DFT(df_bilt2000['temp_mean'])

threshold = 0
mask = np.abs(coefs) > threshold

filtered_k = k[mask]

t_prediction = np.arange(730)
temp_transformed = predict(t_prediction, filtered_k, coefs)

plt.errorbar(df_bilt2000['days'], df_bilt2000['temp_mean'], label='training data', color='tab:blue')
plt.errorbar(df_bilt2001['days'], df_bilt2001['temp_mean'], label='training data', color='tab:orange', fmt='o')

plt.errorbar(t_prediction, temp_transformed, label='model', color='black', fmt='--')
plt.xlim(365, 400)


In [ ]:
plt.plot(k, np.abs(coefs.real))
plt.yscale('log')